# FINN - Analysis Passes
--------------------------------------
This notebook is about analysis passes in FINN. The procedure for creating an analysis pass is shown using an example.

We'll use the following utility functions to print the source code for function calls (`showSrc()`) and to visualize a network using netron (`showInNetron()`) in the Jupyter notebook:

In [4]:
from finn.util.visualization import showSrc, showInNetron

## General Information
------------------------------
* traverses the graph structure and produces information about certain properties
* input: ModelWrapper
* returns dictionary of named properties that the analysis extracts

First the model is shown to illustrate the analysis. For this netron is used. Netron is a visualizer for neural network, deep learning and machine learning models.

In [5]:
model_prefix = "model_sqvgg_4bits_triplet_epoch_-1"

In [36]:
showInNetron(model_prefix+".onnx")

Stopping http://0.0.0.0:8081
Serving 'model_sqvgg_4bits_triplet_epoch_-1.onnx' at http://0.0.0.0:8081


The onnx model has to be converted to a format that can be processed by FINN. This is done with ModelWrapper. As described in the short introduction, this is the format an analysis pass takes as input.

In [37]:
from finn.core.modelwrapper import ModelWrapper
model = ModelWrapper(model_prefix+'.onnx')

The idea is to count all nodes that have the same operation type. The result should contain the operation types and the corresponding number of nodes that occur in the model. In the beginning an empty dictionary is created which is filled by the function and returned as result to the user at the end of the analysis.

In [38]:
def count_equal_nodes(model):
    count_dict = {}
    for node in model.graph.node:
        if node.op_type in count_dict:
            count_dict[node.op_type] +=1
        else:
            count_dict[node.op_type] = 1
    return count_dict

The function takes the model as input and iterates over the nodes. Then it is checked whether there is already an entry for the operation type in the dictionary. If this is not the case, an entry is created and set to `1`. If there is already an entry, it is incremented. If all nodes in the model have been iterated, the filled dictionary is returned.

The analysis function of ModelWrapper is used to perform the analysis just designed. It is shown below and takes the function as input and performs it by passing the model to the function.

In [39]:
showSrc(ModelWrapper.analysis)

    def analysis(self, analysis_fxn):
        """Runs given anaylsis_fxn on this model and return resulting dict."""
        return analysis_fxn(self)



The result can now simply be determined by calling the `.analysis` function.

In [40]:
print(model.analysis(count_equal_nodes))

{'MultiThreshold': 6, 'Add': 2, 'Mul': 13, 'Conv': 5, 'MaxPool': 5, 'Reshape': 1, 'MatMul': 1, 'Div': 1}


In [41]:
import onnx
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
build_dir = "/workspace/finn/build/"
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save(build_dir + model_prefix +"_tidy.onnx")

In [42]:
showInNetron(build_dir+ model_prefix +"_tidy.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/build/model_sqvgg_4bits_triplet_epoch_-1_tidy.onnx' at http://0.0.0.0:8081


In [43]:
print(model.analysis(count_equal_nodes))

{'MultiThreshold': 6, 'Add': 2, 'Mul': 13, 'Conv': 5, 'MaxPool': 5, 'Reshape': 1, 'MatMul': 1, 'Div': 1}


In [44]:
from finn.util.pytorch import ToTensor
from finn.transformation.merge_onnx_models import MergeONNXModels
from finn.core.datatype import DataType

model = ModelWrapper(build_dir+ model_prefix + "_tidy.onnx")
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255

In [45]:
totensor_pyt = ToTensor()
chkpt_preproc_name = build_dir+ model_prefix + "_preproc.onnx"
bo.export_finn_onnx(totensor_pyt, ishape, chkpt_preproc_name)

ir_version: 6
producer_name: "pytorch"
producer_version: "1.7"
graph {
  node {
    input: "0"
    input: "4"
    output: "3"
    name: "Div_0"
    op_type: "Div"
    domain: ""
  }
  name: "torch-jit-export"
  initializer {
    data_type: 1
    name: "4"
    raw_data: "\000\000\177C"
  }
  input {
    name: "0"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 140
          }
          dim {
            dim_value: 140
          }
        }
      }
    }
  }
  input {
    name: "4"
    type {
      tensor_type {
        elem_type: 1
        shape {
        }
      }
    }
  }
  output {
    name: "3"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 140
          }


In [46]:
# join preprocessing and core model
pre_model = ModelWrapper(chkpt_preproc_name)
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT4"])

/workspace/finn-base/src/finn/transformation/infer_data_layouts.py:119: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [47]:
from finn.transformation.infer_datatypes import InferDataTypes
chkpt_name = build_dir+ model_prefix +"_pre_post.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)
showInNetron(chkpt_name)

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/build/model_sqvgg_4bits_triplet_epoch_-1_pre_post.onnx' at http://0.0.0.0:8081


In [63]:
from finn.transformation.streamline import Streamline
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants,MoveMaxPoolPastMultiThreshold
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.general import RemoveUnusedTensors

model = ModelWrapper(chkpt_name)
model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(absorb.AbsorbAddIntoMultiThreshold())
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())
model.save(build_dir+ model_prefix +"_streamlined.onnx")

In [64]:
showInNetron(build_dir+ model_prefix +"_streamlined.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/build/model_sqvgg_4bits_triplet_epoch_-1_streamlined.onnx' at http://0.0.0.0:8081


In [59]:
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.custom_op.registry import getCustomOp
from finn.transformation.infer_data_layouts import InferDataLayouts

# choose the memory mode for the MVTU units, decoupled or const
mem_mode = "decoupled"
model = ModelWrapper(build_dir+ model_prefix +"_streamlined.onnx")
model = model.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode))
# input quantization (if any) to standalone thresholding
model = model.transform(to_hls.InferThresholdingLayer())
model = model.transform(to_hls.InferConvInpGen())
model = model.transform(to_hls.InferStreamingMaxPool())
# get rid of Reshape(-1, 1) operation between hlslib nodes
model = model.transform(RemoveCNVtoFCFlatten())
# get rid of Tranpose -> Tranpose identity seq
model = model.transform(absorb.AbsorbConsecutiveTransposes())
# infer tensor data layouts
model = model.transform(InferDataLayouts())
#parent_model = model.transform(CreateDataflowPartition())



# model = model.transform(to_hls.InferVVAU())
# # input quantization (if any) to standalone thresholding
# model = model.transform(to_hls.InferThresholdingLayer())
# model = model.transform(to_hls.InferConvInpGen())
# model = model.transform(to_hls.InferStreamingMaxPool())
# model = model.transform(to_hls.InferPool_Batch())

# model = model.transform(RemoveCNVtoFCFlatten())
# model = model.transform(absorb.AbsorbConsecutiveTransposes())
# # infer tensor data layouts
# model = model.transform(InferDataLayouts())
# parent_model = model.transform(CreateDataflowPartition())
# # 
# # model = model.transform(RemoveCNVtoFCFlatten())

model.save(build_dir+ model_prefix +"_dataflow.onnx")
showInNetron(build_dir+ model_prefix +"_dataflow.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/build/model_sqvgg_4bits_triplet_epoch_-1_dataflow.onnx' at http://0.0.0.0:8081


In [62]:
test_pynq_board = "Pynq-Z2"
target_clk_ns = 10
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild

model = model.transform(ZynqBuild(platform = test_pynq_board, period_ns = target_clk_ns))
model.save(build_dir + "/end2end_cnv_w1a1_synth.onnx")

AttributeError: 'NoneType' object has no attribute 's'